# Tutorial on manipulating MSI L1C product


In [ ]:
from pathlib import Path

import cartopy.crs as ccrs  # For static plotting
import cartopy.feature as cf
import datatree as dt
import geopandas  # For interactive plotting
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

## Inputs

### Get AWS credentials

In [ ]:
import json
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s2input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s2input"].pop("region_name",None)


### Browse S3 bucket and get MSIL1C product

In [ ]:
import s3fs
SAMPLE_PATH = "s3://s2-input/Products/S2MSI--L0plus-L1A-L1B-L1C-L2A/"
s3 = s3fs.S3FileSystem(
    key=secrets["s2input"]["key"],
    secret=secrets["s2input"]["secret"],
    endpoint_url=secrets["s2input"]["endpoint_url"]
)
s3_path = s3.glob(SAMPLE_PATH+"S2MSIL1C*.zip")
store=f"zip::s3://{s3_path[0]}"
store

### Open the product


In [ ]:
xdt = dt.open_datatree(store, engine="zarr", mode="r", chunks={},backend_kwargs={"storage_options": {"s3":secrets["s2input"]}})
xdt

### Overview of the product content

In [ ]:
xdt["conditions/detfoo/r60m"]["b01"].plot()

In [ ]:
xdt["measurements/r60m"]["b01"].plot()

In [ ]:
xdt["measurements/r60m"]["b01"].encoding["add_offset"]

In [ ]:
xdt["measurements/r60m"]["b01"].max(), xdt["measurements/r60m"]["b01"].min(), xdt[
    "measurements/r60m"
]["b01"].mean()

In [ ]:
r = xdt["measurements/r10m"]["b04"]
g = xdt["measurements/r10m"]["b03"]
b = xdt["measurements/r10m"]["b02"]

xr.Dataset(dict(r=r, g=g, b=b))

In [ ]:
b02 = xdt["measurements/r10m"]["b02"]
b02

In [ ]:
b02.dtype

In [ ]:
b02.encoding

## Plot a RGB image

In [ ]:
rgb_band_paths = (
            f"measurements/r10m/b04",
            f"measurements/r10m/b03",
            f"measurements/r10m/b02",
        )

concat = xr.concat(
        [xdt[str(p)] for p in rgb_band_paths],  # type: ignore
        dim="band",
    )

ax = concat.plot.imshow()
ax.axes.set_aspect("equal")
plt.gca().invert_yaxis()

### Explore product geolocation

The following snippet shows an interactive map with the tile's footprint


#### Interactive map

In [ ]:
gdf = geopandas.GeoDataFrame.from_features([xdt.attrs["stac_discovery"]])

Note: CRS is missing from the metadata, it must be set manually


In [ ]:
gdf = gdf.set_crs(4326)

In [ ]:
gdf.explore()

In [ ]:
gdf.crs

#### Non-interactive map

The following snippet shows the location of the tile on a global map.


In [ ]:
def main():
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())

    # make the map global rather than have it zoom in to
    # the extents of any plotted data
    ax.set_global()

    # ax.stock_img()
    ax.coastlines()

    ax.plot(gdf.centroid[0].x, gdf.centroid[0].y, "ro", transform=ccrs.PlateCarree())

    plt.show()


if __name__ == "__main__":
    main()

#### Plot georeferenced data

In [ ]:
# Define constant for plotting
L1C_PROJECTION = ccrs.epsg(32633)
DESIRED_PROJECTION = ccrs.PlateCarree()
FIGSIZE: tuple[int, int] = (12, 8)
RESOLUTION_CARTOPY: str = '110m'
GEOGRAPHICAL_LIMITS: tuple[int, int, int, int] = (-20, 30, 10, 30)
GEOGRAPHICAL_LIMITS: tuple[int, int, int, int] = (0, 10, 42, 46)

# Speed up plot by sampling data every SKIP_EVERY pixels
SKIP_EVERY: int = 50

# Define plotting arguments for Polygon around the area of interest
POLYGON_THICKNESS: int = 1
POLYGON_COLOR: str = 'r'

# Get the geometry from the product and check that it correspond to the domain represented
geometry_from_product = np.squeeze(xdt.attrs["stac_discovery"]["geometry"]["coordinates"])
geometry_from_product

In [ ]:
_, ax = plt.subplots(subplot_kw={"projection": DESIRED_PROJECTION},
                    figsize=FIGSIZE)

# Plot cartopy geographic information
ax.coastlines(resolution=RESOLUTION_CARTOPY)
ax.add_feature(cf.BORDERS)
ax.add_feature(cf.OCEAN)
ax.add_feature(cf.LAND)
gl = ax.gridlines(draw_labels=True, 
                  crs=DESIRED_PROJECTION)


b02 = xdt["measurements/r10m"]["b02"]
plt.contourf(b02[::SKIP_EVERY, ::SKIP_EVERY], transform=L1C_PROJECTION)
poly = mpatches.Polygon(geometry_from_product, 
                        closed=True, 
                        ec=POLYGON_COLOR, 
                        fill=False, 
                        lw=POLYGON_THICKNESS, 
                        transform=DESIRED_PROJECTION)
ax.add_patch(poly)
ax.set_extent(GEOGRAPHICAL_LIMITS, crs=DESIRED_PROJECTION)
cbar = plt.colorbar(orientation="horizontal")
cbar.set_label('b02_10m')
plt.tight_layout()

## Compute radiances

From: https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-1c

$radiance = reflectance * \cos(radians(SunZenithAngle)) * solarIrradiance * U / pi$

In [ ]:
U: float = xdt.attrs["other_metadata"]["reflectance_correction_factor_from_the_Sun-Earth_distance_variation_computed_using_the_acquisition_date"]
U

In [ ]:
# Be carefull, Sun Zenith Angle is expressed on the angles grid (5km), it needs to be reprojected on the 10m grid for computing radiances
# cosinus is applied now because we can not interpolate angles using a linear interpolation (discontinuity at 0°)
# On the other hand, cosines can be interpolated
# cos_zsa_5km: EOVariable = np.cos(np.deg2rad(xdt[].conditions.geometry.sza))
# cos_zsa_5km
cos_sza_5km: xr.DataArray = np.cos(np.deg2rad(xdt["conditions/sun_ang/zen"]))
cos_sza_5km

In [ ]:
# We will convert reflectances from band BAND to radiances
BAND: int = 2

# Band - 1 because Python list index starts at 0
solarIrradiance: float = np.float64(xdt.attrs["stac_discovery"]["properties"]["eo:bands"][BAND-1]["solar_illumination"])

In [ ]:
reflectance_b02_10m: xr.DataArray = xdt["measurements/r10m/b02"]
reflectance_b02_10m

In [ ]:
# Interpolate sza on the angles grid to the 10m grid
cos_sza_10m = cos_sza_5km.interp_like(reflectance_b02_10m)
cos_sza_10m

In [ ]:
# For simplicity, radiance computation assume that reflectances equal numerical counts
radiance = reflectance_b02_10m * cos_sza_10m * solarIrradiance * U / np.pi
radiance

Visualize computational graph
(Needs to have graphviz package)

In [ ]:
radiance.data.visualize()

In [ ]:
radiance.plot()